# Kapitel 7 - Hyperparameteroptimierung

## 7.1. Kapitelübersicht <a class="anchor" id="7-1"/>

Bis jetzt hatten wir nur die Standardhyperparameter von Scikit learn bei unseren Klassifizierungsverfahren genutzt. Durch eine gezielte <b>Optimierung</b> der Hyperparameter können Klassifizierungsverfahren jedoch deutlich verbessert werden. Für diese Suche nach den optimalen Hyperparametern existieren spezielle Suchverfahren wie <b>GridSearch</b> oder <b>RandomSearch</b>, die wir uns in diesem Kapitel genauer anschauen werden. Wir werden sie benutzen, um die Klassifizierungsverfahren Multinomial Naive Bayes und Logistic Regression zu verbessern. 


<b>Abschnittsübersicht</b><br>

[7.1. Kapitelübersicht](#7-1)<br>
[7.2. Hyperparameter von Multinomial Naive Bayes](#7-2)<br>
[7.3. Grid Search](#7-3)<br>
[7.4. Random Search](#7-4)<br>
[7.6. Hyperparameteroptimierung von Logistic Regression](#7-5)<br>
[7.6. Mögliche Fehler](#7-6)<br>

## 7.2. Hyperparameter von Multinomial Naive Bayes <a class="anchor" id="7-2"/>

In [3]:
import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


Implementieren wir zunächst ein weiteres Mal das <b>Multinomial Naive Bayes</b> Klassifizierungsverfahren.

In [54]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import numpy as np


labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer().fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

def classify_mnb(alpha=1.0):   
    # Multinomial Naive Bayes
    mnb_classifier = MultinomialNB(alpha)
    mnb = mnb_classifier.fit(X_train, y_train)

    # cross validation des Trainingsdatensatzes
    mnb_scores = cross_val_score(mnb_classifier, vector, labels, cv=3)
    mnb_mean = np.mean(mnb_scores)

    print("Der Mittelwert der cross validation bei der  Klassifizierung " 
          + f" mit Multinomial Naive Bayes ist {str(np.around(mnb_mean, decimals=3))}."
          + "\n")


    # F1-score des Testdatensatzes
    y_pred = mnb_classifier.predict(X_test)
    mnb_f1 = f1_score(y_test, y_pred, average="micro")

    print("Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist "
          + f"{str(np.around(mnb_f1, decimals=3))}.")
print(classify_mnb())

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.859.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.87.
None


Schauen wir uns die möglichen Parameter von Multinomial Naive Bayes in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html">Dokumentation</a> an. Es gibt drei Parameter:
- `alpha` (default = 1.0)
- `fit_prior` (default = True)
- `class_prior` (default = None)<br>

Wir werden uns hier nur `alpha` anschauen. Ist `alpha = 1`, wird <b>Laplace Smoothing</b> angewandt (siehe Kapitel 3), d.h. jede Worthäufigkeit wird um 1 erhöht. Ist `alpha < 1`, wird <b>Lidstone Smoothing</b> angewandt, welches im Grunde das Gleiche ist. Ist `alpha = 0`, wird gar kein Smoothing angewandt. Standardmäßig ist `alpha = 1` ausgewählt, weshalb wir hier zunächst `alpha = 0.5` setzen.

In [55]:
classify_mnb(0.5)

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.876.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.889.


Sowohl der Mittelwert der cross validation als auch der F1-score haben sich verbessert. Versuchen wir es nun mit `alpha = 0.1`.

In [56]:
classify_mnb(0.1)

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.898.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.906.


Erneut können wir eine Verbesserung feststellen. Um weitere Parameter zu testen, können wir nun zwei Strategien verfolgen: Parameter durch Ausprobieren zu optimieren oder ein Suchverfahren zu benutzen, um die optimalen Hyperparameter zu finden.

## 7.3. Grid Search <a class="anchor" id="7-3"/>

<b>Grid Search</b> (deutsch: Rastersuche) ist ein Suchverfahren, welches mithilfe der <b>Brute-Force-Methode</b> (auch <i>erschöpfende Suche</i> genannt) die optimalen Hyperparameter sucht. Dabei muss ein selbst erstelltes Dictionary von möglichen Hyperparametern als keys übergeben werden (hier `parameters`), deren values mögliche Parameterwerte sind. Grid Search nutzt als Evaluationsmethode die <b>cross validation</b>, weshalb man die Anzahl der Teildatensätze (<i>folds</i>) beim Parameter `cv` angeben muss. Zudem müssen wir, um den <b>F1-score nutzen</b> zu können, diesen beim Parameter `scoring` angeben. Grid Search liefert gute Werte, leidet aber unter dem <b>Fluch der Dimensionalität</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Fluch der Dimensionalität
    
Der <b>Fluch der Dimensionalität</b> (englisch: curse of dimensionality) ist ein Begriff, der auf die Schwierigkeiten beim Anpassen von Modellen, bei der Schätzung von Parametern oder bei der Optimierung einer Funktion in vielen Dimensionen verweisen soll. Je mehr Dimensionen ein Eingabedatenraum hat, umso schwieriger wird es, optimale Parameter für diesen Raum zu finden. Bei den Klassifizierungsverfahren bedeutet dies konkret: Je mehr Hyperparameter ein Klassifizierungsverfahren hat, umso schwieriger wird es, diese zu optimieren, um bspw. einen idealen F1-score zu erreichen.

In [57]:
from sklearn.model_selection import GridSearchCV

parameters = {"alpha": np.array([0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 
                0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])}

grid = GridSearchCV(MultinomialNB(), parameters, cv=5, scoring="f1_micro")
grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für alpha ist {str(grid.best_estimator_.alpha)}.")
print(f"Der beste Score ist {str(np.around(grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für alpha ist 0.01.
Der beste Score ist 0.9302.
CPU times: user 33.8 s, sys: 2.36 s, total: 36.1 s
Wall time: 36.1 s


Von unseren Hyperparametern scheint `0.01` der beste Hyperparameter zu sein. Wir können nun die umliegenden Werte zu `0.01` als neue Parameter auswählen und GridSearch übergeben.

In [10]:
parameters = {"alpha": np.array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006,
                                 0.007, 0.008, 0.009, 0.01, 0.011, 0.012,
                                0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 
                                0.019, 0.2])}

grid = GridSearchCV(MultinomialNB(), parameters, cv=5, scoring="f1_micro")
grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für alpha ist {str(grid.best_estimator_.alpha)}.")
print(f"Der beste Score ist {str(np.around(grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für alpha ist 0.006.
Der beste Score ist 0.9317.


Ein noch besserer Wert als `0.01` scheint `0.006` zu sein, der Score hat sich von `0.9302` auf `0.9317` erhöht. Bei der Hyperparameteroptimierung sollte man sich immer fragen, ob sich der Aufwand für eine so geringe Verbesserung lohnt. Eine eindeutige Antwort gibt es darauf nicht, eine Verbesserung anzustreben, die kleiner als `0.01` ist, ist für unsere Zwecke nicht sonderlich sinnvoll. Allgemein könnte man festlegen, dass man wenn möglich optimieren sollte, aber nur, wenn diese Optimierung auch sinnvoll ist. 

> Premature optimization is the root of all evil. - Tony Hoare

## 7.4. Random Search <a class="anchor" id="7-4"/>

Bei <b>Random Search</b> (deutsch: Zufallssuche) werden die optimalen Hyperparameter anders als bei <b>Grid Search</b> nicht durch erschöpfendes Ausprobieren aller Kombinationen gefunden, sonderen durch eine zufällige Auswahl von Parametern bei einer vorgegeben Anzahl an Durchläufen. Die Anzahl der Durchläufe können wir mit `n_iter` festlegen. Mit `uniform(0,1)` erzeugen wir zufällige Zahlen zwischen 0 und 1. Wir haben hier `n_iter = 10` gesetzt, für bessere Ergebnisse sollte `n_iter` höher gesetzt werden, die Ausführung dauert dann entsprechend länger.

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform 

param_grid = {'alpha': uniform(0, 1)}
rsearch = RandomizedSearchCV(MultinomialNB(), param_grid, n_iter=10, cv=5)
rsearch.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für alpha ist {str(rsearch.best_estimator_.alpha)}.")
print(f"Der beste Score ist {str(np.around(rsearch.best_score_, decimals=4))}.")

Der beste Hyperparameter für alpha ist 0.07519616874634016.
Der beste Score ist 0.915.


## 7.5. Hyperparameteroptimierung von Logistic Regression <a class="anchor" id="7-5"/>

Zunächst implementieren wir <b>Logistic Regression</b>.

In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

import numpy as np


labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer().fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)


# Logistic Regression
lr_classifier = LogisticRegression()
lr = lr_classifier.fit(X_train, y_train)

# cross validation des Trainingsdatensatzes
lr_scores = cross_val_score(lr_classifier, vector, labels, cv=3)
lr_mean = np.mean(lr_scores)

print("Der Mittelwert der cross validation bei der  Klassifizierung " 
      + f" mit Logistic Regression ist {str(np.around(lr_mean, decimals=3))}."
      + "\n")


# F1-score des Testdatensatzes
y_pred = lr_classifier.predict(X_test)
lr_f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung mit Logistic regression ist "
      + f"{str(np.around(lr_f1, decimals=3))}.")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jan/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the m

Der Mittelwert der cross validation bei der  Klassifizierung  mit Logistic Regression ist 0.904.

Der F1-score für die Klassifizierung mit Logistic regression ist 0.934.


Schauen wir uns die möglichen Parameter von Logistic Regression in der <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">Dokumentation</a> an. Die Anzahl der möglichen Hyperparameter ist um einiges größer als bei Multinomial Naive Bayes:
- `penalty` (default='l2')
- `dual` (default=False)
- `tol` (default=1e-4)
- `C` (default=1.0)
- `fit_intercept` (default=True)
- `intercept_scaling` (default=1)
- `class_weight` (default=None)
- `random_state` (default=None)
- `solve` (default='liblinear')
- `max_iter` (default=100)
- `multi_class` (default='ovr')
- `verbose` (default=0)
- `warm_start` (default=False)
- `n_jobs` (default=None)
- `l1_ratio` (default=None)

Anders als bei deim Multinomial Naive Bayes Verfahren gibt es hier nicht nur viel mehr Parameter, manche Parameter funktionieren nur für die binäre Klassifikation, andere nur für die Multiclass Klassifikation. Viele Parameter sind zudem von der Wahl anderer Parameter abhängig. Vom Parameter `solver` sind die Parameter `penalty`, `dual`, `verbose`, `warm_start` oder `l1_ratio` direkt oder indirekt (abhängig von einem Parameter, der von `solver` abhängig ist) abhängig. `solver` kann folgende Werte annehmen(?): `newton-cg`, `lbfgs`, `liblinear`, `sag` oder `saga`. `liblinear` funktoniert bei kleinen Datensätzen besser, `sag` und `saga` bei großen Datensätzen. `liblinear` behandelt nicht wie die anderen Parameter den Multinomial Loss, d.h. es kann beim Hyperparameter `multi_class` nicht der Wert `multinomial` benutzt werden.<br>

Wir haben nun verschiedene Werte für den `C` Parameter verwendet und den `solver` auf `lbfgs` gesetzt. <b>Achtung</b>: Die Ausführung dauert so etwa 4-8 Minuten.

In [50]:
parameters = {"C": [0.1, 1, 2],
              "penalty":['l2'],
              "multi_class": ["ovr"],
              "solver": ["lbfgs"]}


lr_grid = GridSearchCV(LogisticRegression(), parameters, cv=3, scoring="f1_micro")
lr_grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für C ist {str(lr_grid.best_estimator_.C)}.")
print(f"Der beste Score ist {str(np.around(lr_grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für C ist 2.
Der beste Score ist 0.9238.
312.8535635471344


<div class="alert alert-warning">
<b>Aufgabe:</b> Hyperparameteroptimierung von Logistic Regression
    
Versuchen Sie mit dem <b>Grid Search</b> Suchverfahren optimale Hyperparameter für Logistic Regression zu finden. Passen Sie dafür nur folgende Parameter an:
- `C` (Tipp: nicht zu viele verschiedene Werte nehmen, max. 7)
- `solver` (Tipp: nicht `liblinear` wählen)
- `multi_class`

<b>Achtung</b>: Die Berechnungszeit kann sehr lange werden (AF?). Um die Berechnungszeit zu verkürzen, setzen Sie `cv = 3`. Eine weitere Möglichkeit, die Berechnung zu beschleunigen, ist die Nutzung des Hyperparameters `n_jobs`. Wenn `multi_class = 'ovr'` ausgewählt ist, können mehrere Kerne für die Ausführung genutzt werden, wobei `n_jobs = -1` alle verfügbaren Kerne benutzt. Andere Anwendungen auf dem ausführenden Computer können so aber stark verlangsamt werden. 

#### Mögliche Lösung

Hier eine mögliche Lösung. Die Ausführung hat über zwei Stunden gedauert. Der cross validation Wert konnte von 0.904 auf 0.935 verbessert werden.

In [59]:
%%time
parameters = {"C": [2, 3, 4, 5, 6, 7, 8, 9, 10],
              "penalty":['l2'],
              "multi_class": ["ovr", "multinomial"],
              "solver": ["lbfgs", "newton-cg"]}


lr_grid = GridSearchCV(LogisticRegression(), parameters, cv=3, scoring="f1_micro")
lr_grid.fit(X_train, y_train)

# Ergebnisse
print(f"Der beste Hyperparameter für C ist {str(lr_grid.best_estimator_.C)}.")
print(f"Der beste Hyperparameter für multi_class ist {str(lr_grid.best_estimator_.multi_class)}.")
print(f"Der beste Hyperparameter für solve ist {str(lr_grid.best_estimator_.solver)}.")
print(f"Der beste Score ist {str(np.around(lr_grid.best_score_, decimals=4))}.")

Der beste Hyperparameter für C ist 8.
Der beste Hyperparameter für multi_class ist ovr.
Der beste Hyperparameter für solve ist lbfgs.
Der beste Score ist 0.935.
CPU times: user 5h 43min 21s, sys: 3h 51min 8s, total: 9h 34min 29s
Wall time: 2h 3min 8s


## 7.6. Mögliche Fehler <a class="anchor" id="7-6"/>

<b>GridSearch</b>:
- F1-score soll genutzt werden, `scoring` Parameter wurde nicht definiert → `scoring`-Parameter angeben mit Wert "f1", "f1_micro", "f1_macro" etc.
- Fehlercode: `Target is multiclass but average='binary'. Please choose another average setting.` → Nur "f1" wurde beim Parameter `scoring` angegeben, obwohl die es mehr als zwei Klassen bei den Daten gibt. Standardmäßig geht "f1" von einer binären Klassifikation aus, deshalb muss z.B. "f1_micro" oder "f1_macro" angegeben werden.
- Ausführung von RandomSearch dauert sehr lange → `n_iter` heruntersetzen.